In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/06 09:10:12 WARN Utils: Your hostname, ROG-STRIX resolves to a loopback address: 127.0.1.1; using 172.27.115.120 instead (on interface eth0)
23/03/06 09:10:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 09:10:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [4]:
for key, value in df_green.rdd.take(1)[0].asDict().items():
    print(f"{key}: {value}")

VendorID: 2
lpep_pickup_datetime: 2020-01-14 15:01:42
lpep_dropoff_datetime: 2020-01-14 15:28:14
store_and_fwd_flag: N
RatecodeID: 1
PULocationID: 129
DOLocationID: 141
passenger_count: 5
trip_distance: 5.16
fare_amount: 20.5
extra: 0.0
mta_tax: 0.5
tip_amount: 0.0
tolls_amount: 0.0
ehail_fee: None
improvement_surcharge: 0.3
total_amount: 24.05
payment_type: 2
trip_type: 1
congestion_surcharge: 2.75


In [5]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [6]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 15, 1, 42), PULocationID=129, total_amount=24.05),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 8, 8, 24), PULocationID=18, total_amount=10.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 54), PULocationID=41, total_amount=14.54),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 13, 22, 44), PULocationID=41, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 26, 11, 45, 13), PULocationID=116, total_amount=4.3)]

In [7]:
from datetime import datetime

In [8]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [9]:
rows = rdd.take(10)
row = rows[0]

In [10]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 14, 15, 1, 42), PULocationID=129, total_amount=24.05)

In [11]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [12]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(10)

[((datetime.datetime(2020, 1, 14, 15, 0), 129), (24.05, 1)),
 ((datetime.datetime(2020, 1, 7, 8, 0), 18), (10.8, 1)),
 ((datetime.datetime(2020, 1, 12, 18, 0), 41), (14.54, 1)),
 ((datetime.datetime(2020, 1, 3, 13, 0), 41), (8.3, 1)),
 ((datetime.datetime(2020, 1, 26, 11, 0), 116), (4.3, 1)),
 ((datetime.datetime(2020, 1, 31, 13, 0), 164), (19.67, 1)),
 ((datetime.datetime(2020, 1, 9, 20, 0), 260), (10.3, 1)),
 ((datetime.datetime(2020, 1, 9, 18, 0), 66), (8.3, 1)),
 ((datetime.datetime(2020, 1, 7, 11, 0), 33), (27.0, 1)),
 ((datetime.datetime(2020, 1, 27, 13, 0), 174), (25.5, 1))]

In [21]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [14]:
from collections import namedtuple

In [15]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [16]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [17]:
from pyspark.sql import types

In [18]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [22]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [25]:
df_result.write.parquet('tmp/green-revenue', mode="overwrite")

23/03/06 13:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/06 13:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/06 13:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/06 13:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/06 13:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/06 13:08:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/06 13:08:33 WARN MemoryManager: Total allocation exceeds 95.

## Spark RDD mapPartition

In [26]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [27]:
import pandas as pd

In [28]:
rows = duration_rdd.take(10)

In [29]:
df = pd.DataFrame(rows, columns=columns)

In [30]:
df.head()

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2.0,2020-01-14 15:01:42,129,141,5.16
1,2.0,2020-01-07 08:08:24,18,241,2.15
2,NaN,2020-01-12 18:54:00,41,235,4.62
3,2.0,2020-01-03 13:22:44,41,75,1.15
4,2.0,2020-01-26 11:45:13,116,116,0.43


In [32]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [33]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [34]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [35]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|              25.8|
|             10.75|
|              23.1|
|              5.75|
|              2.15|
|7.8500000000000005|
|               7.1|
|              6.05|
|               0.0|
|14.350000000000001|
|              51.2|
|               5.5|
|17.150000000000002|
|               5.0|
|               4.5|
|              16.8|
| 9.549999999999999|
|              10.0|
|             24.15|
|10.649999999999999|
+------------------+
only showing top 20 rows

